## Обзор статьи: Revisiting Deep Learning Models for Tabular Data

В недавней статье "Revisiting Deep Learning Models for Tabular Data" от Яндекса ([Gorishniy et al., 2021]($Revisiting Deep Learning Models for Tabular Data$), см. также [презентацию](https://youtu.be/ObzrXjqWcTY?t=1445)) авторы сравнивали эффективность различных моделей машинного обучения на табличных данных. При этом:

1. Сравнивались 11 моделей, в том числе XGBoost, CatBoost, MLP, [ResNet](https://github.com/yandex-research/rtdl/blob/main/bin/resnet.py), [FT-Transformer](https://github.com/yandex-research/rtdl/blob/main/bin/ft_transformer.py).
2. Для сравнения использовались 11 датасетов для классификации и регрессии, в том числе California Housing, Adult income estimation, Higgs, Covertype и другие. Датасеты имели от 20 тысяч до 1.2 миллиона примеров и от 8 до 2000 признаков. 
3. Использовалось только одно (одинаковое для всех моделей) деление датасетов на обучающую, валидационную и тестовую части. Подбор гиперпараметров осуществлялся на валидационной части.
4. Модели сравнивались сначала с параметрами по-умолчанию, затем осуществлялся подбор гиперпараметров методом TPE (Tree-structured Parzen Estimator) с помощью библиотеки Optuna, при этом количество итераций подбора было одинаковым для всех моделей (время подбора могло различаться).
5. Точность моделей оценивалась по средней метрике качества по 15 попыткам обучения. Полученные 15 моделей ансамблировались в три группы по 5 моделей, и метрика усреднялась по трем ансамблям. В результате для каждой модели были получены средняя точность одиночной модели и средняя точность ансабмля из 5 моделей.

В результате на разных датасетах показали наилучшее качество разные модели.

- Среди *одиночных моделей* FT-Transformer достиг наилучшей точности на 4 из 11 датасетов, остальные модели не более чем на 2 из 11 датасетов. На 7 из 11 датасетах наилучшей точности достигла одна из  нейросетевых моделей.
- После *ансамблирования* FT-Transformer показывает наилучшую точность на 5 из 11 датасетах, CatBoost на 3 из 11 датасетах, остальные модели - не более чем на одном датасете. Снова на 7 из 11 датасетах (тех же самых) наилучшей точности достигла одна из  нейросетевых моделей.

Также все 11 моделей были протестированы на ряде [синтетических датасетов](https://github.com/yandex-research/rtdl/blob/main/lib/synthetic_data.py), которые были получены по формуле $y = \alpha f_\text{GBDT}(x) + (1-\alpha) f_\text{DL}(x)$, где $f_\text{GBDT}$ - ансамбль из 30 случайных решающих деревьев и $f_\text{DL}$ - ансамбль из 30 MLP-сетей с тремя случайно инициализированными скрытыми слоями.

На этих данных авторы сравнили три архитектуры: GBDT, ResNet и FT-Transformer. Естественно, что чем больше $\alpha$, тем хуже обобщался на этих модельных данных ResNet и лучше обобщался градиентный бустинг. При этом получен интересный результат: FT-Transformer обобщался почти не хуже решающих деревьев и почти не хуже ResNet при всех $\alpha$.

<img src="assets/ft.png" width="400" align="center">

Конечно, данный бенчмарк нельзя назвать исчерпывающим. Например, категориальные признаки присутствовали лишь в одном из 11 реальных датасетов, также не было датасетов с небольшим числом примеров. Но дело не только в этом. В данном бенчмарке рассчитывается только усредненная метрика качества при тестировании (RMSE, accuracy). Это может помочь найти наилучшие модели, но не поможет понять причину того, что одни модели обобщаются лучше других. Чтобы разобраться в этом вопросе, нужно более детально изучать обучение и поведение модели.

## Способы анализа нейронных сетей и решающих деревьев

Перечисленные ниже способы могут помочь понять причину того, что они модели обобщаются лучше или хуже других.

**Первый способ.** Обучить модель многократно на подвыборках разного размера из некоего большого датасета. Так мы сможем измерить систематическую (bias) и несистематическую (variance) ошибку модели.

**Второй способ.** Изучение свойств функции, реализуемой моделью, до и после обучения. Для этого можно взять один тестовый пример и проследить, как меняется предсказание модели при изменении того или иного признака (двухмерный график) или сразу двух признаков (трехмерный график). Также можно взять два или три тестовых примера и проследить, как меняется предсказание модели при плавном переходе между ними.

- Вид полученных функций поможет лучше понять общие свойства функции от многих переменных, реализуемой моделью, так как в данном способе мы изучаем срезы этой функции. 
- Сравнение функций до и после обучения может помочь понять, как модель обучается и какие решения среди всего пространства поиска предпочитает алгоритм обучения, а каких решений избегает (даже если они есть в пространстве поиска).

Идеальной была бы ситуация, когда мы имеем в явном виде целевую функцию, и тогда мы сможем сравнить полученные функции с верными. Однако целевую функцию в явном виде мы имеем только на синтетических датасетах, работа моделей на которых может быть не показательной, так как они отличаются от реальных. На реальных данных хорошую аппроксимацию целевой функции можно получить, обучив ансамбль из моделей разных архитектур на большом датасете. Полученную функцию можно считать близкой к эталонной, и с ней сравнивать функции, реализуемые моделями, обученными на существенно меньшем объеме данных.

**Третий способ.** Кроме расчета усредненной метрики качества по всей тестовой выборке, можно измерить метрику на отдельных типах примеров, например одним из следующих способов:
- Разбив выборку на части по значению одного из исходных признаков.
- Разбив выборку на части по значению целевого признака.
- Выполнив кластеризацию выборки по SHAP values.

Таким образом мы сможем выявить ситуацию, когда модель $A$ дает качество лучше, чем модель $B$ на большинстве примеров, но сильную ошибку на отдельных типах примеров, из-за чего усредненная метрика качества у модели $B$ оказывается выше.

## Далее...

Часть 1.

Boston housing: нейросети до и после обучения

https://www.cs.toronto.edu/~radford/ftp/pin.pdf

Решающие деревья после обучения

Часть 2.

Mixture model: сумма функций от одного признака, в сравнении с решающими деревьями глубины 1.

Нейросети предпочитают определенные классы решений, хотя пространство поиска содержит почти все возможные функции.

Как изменить класс решений, предпочитаемый сетью?

Связь с deep image prior.